In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense
from keras.models import Sequential

import glob, os, random

Using TensorFlow backend.


In [14]:
base_path = 'C:/Users/pujit/Downloads/CNN/data'
#base_path = '/home/014505660/waste_seg/CNN/data'

img_list = glob.glob(os.path.join(base_path, '*/*.png'))

print(len(img_list))

9600


In [15]:
'''
for i, img_path in enumerate(random.sample(img_list, 6)):
    img = load_img(img_path)
    img = img_to_array(img, dtype=np.uint8)

    plt.subplot(2, 3, i+1)
    plt.imshow(img.squeeze())
'''

'\nfor i, img_path in enumerate(random.sample(img_list, 6)):\n    img = load_img(img_path)\n    img = img_to_array(img, dtype=np.uint8)\n\n    plt.subplot(2, 3, i+1)\n    plt.imshow(img.squeeze())\n'

In [16]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.1
)

In [17]:
test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)


In [18]:
train_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=(384, 512),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    seed=0
)

Found 8640 images belonging to 6 classes.


In [19]:
validation_generator = test_datagen.flow_from_directory(
    base_path,
    target_size=(384, 512),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

Found 960 images belonging to 6 classes.


In [20]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [21]:
model = Sequential([
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(384, 512, 3)),
    MaxPooling2D(pool_size=2),

    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),

    Flatten(),

    Dense(64, activation='relu'),

    Dense(6, activation='softmax')
])

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 384, 512, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 192, 256, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 192, 256, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 96, 128, 64)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 96, 128, 32)       18464     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 48, 64, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 48, 64, 32)       

In [23]:
model.fit_generator(train_generator, epochs=20, validation_data=validation_generator, validation_steps=20)

Epoch 1/20
425/540 [======================>.......] - ETA: 8:07 - loss: 1.6030 - acc: 0.3232

KeyboardInterrupt: 